In [1]:
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import loggingService
import os
from sentence_transformers import SentenceTransformer
import weaviate

load_dotenv()
logger = loggingService.get_logger()


In [2]:
apikey = os.getenv("GEANAI_KEY", None)
class_name = os.getenv("WEVIATE_CLASS", 'Livros')
path = os.getenv("DATA_PATH", "data")
model_name = os.getenv("MODEL_NAME_EMBEDDING", "sentence-transformers/gtr-t5-large")
weaviate_url = os.getenv("WEAVIATE_URL", 'http://127.0.0.1:8080')

client = weaviate.Client(
    url=weaviate_url,
)

embeddings = []
pages = []
pdf_loader = PyPDFDirectoryLoader(path)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50, separators=["\n\n", "\n", " ", "", "\n \n", "\n \n \n","\n\n\n"])
documents = pdf_loader.load_and_split(text_splitter=text_splitter)
print(len(documents))
print(documents[0])

2199
page_content='Memórias Póstumas de Brás Cubas, Machado de Assis \n \nFonte: \nAssis, Machado de. Obra Completa. vol. I. Rio de Janeiro: Nova Aguilar, 1994. \n \nTexto proveniente de: \nA Biblioteca Virtual do Estudante Brasileiro <http://www.bibvirt.futuro.usp.br> \nA Escola do Futuro da Universidade de São Paulo \nPermitido o uso apenas para fins educacionais. \n \nTexto-base digitalizado por: \nNúcleo de Pesquisas em Informática, Literatura e Lingüística \n<http://www.cce.ufsc.br/~nupill/literatura/literat.html> \n \nEste material pode ser redistribuído livremente, desde que não seja alterado, e que as informações \nacima sejam mantidas. Para maiores informações, escreva para <bibvirt@futuro.usp.br>. \n \nEstamos em busca de patrocinadores e voluntários para nos ajudar a manter este projeto. Se você \nquer ajudar de alguma forma, mande um e-mail para <bibvirt@futuro.usp.br> e saiba como isso é \npossível. \n \n \n \nMEMÓRIAS PÓSTUMAS DE BRÁS CUBAS \nMACHADO DE ASSIS \n \n \n    

In [6]:
def pdf_text_splitter(pdf_text) -> dict:
  retorno = {'content': '', 'source': '', 'page': 0}
  
  retorno['content'] = getattr(pdf_text, 'page_content')
  retorno['souce'] = getattr(pdf_text, 'metadata')['source']
  retorno['page'] = getattr(pdf_text, 'metadata')['page']
  
  return retorno

for doc in documents:
  # logger.debug(pdf_text_splitter(doc))
  # print(pdf_text_splitter(doc))
  pages.append(pdf_text_splitter(doc))
  
print(pages[0])

{'content': 'Memórias Póstumas de Brás Cubas, Machado de Assis \n \nFonte: \nAssis, Machado de. Obra Completa. vol. I. Rio de Janeiro: Nova Aguilar, 1994. \n \nTexto proveniente de: \nA Biblioteca Virtual do Estudante Brasileiro <http://www.bibvirt.futuro.usp.br> \nA Escola do Futuro da Universidade de São Paulo \nPermitido o uso apenas para fins educacionais. \n \nTexto-base digitalizado por: \nNúcleo de Pesquisas em Informática, Literatura e Lingüística \n<http://www.cce.ufsc.br/~nupill/literatura/literat.html> \n \nEste material pode ser redistribuído livremente, desde que não seja alterado, e que as informações \nacima sejam mantidas. Para maiores informações, escreva para <bibvirt@futuro.usp.br>. \n \nEstamos em busca de patrocinadores e voluntários para nos ajudar a manter este projeto. Se você \nquer ajudar de alguma forma, mande um e-mail para <bibvirt@futuro.usp.br> e saiba como isso é \npossível. \n \n \n \nMEMÓRIAS PÓSTUMAS DE BRÁS CUBAS \nMACHADO DE ASSIS \n \n \n          

In [7]:
print(len(pages))
model = SentenceTransformer(model_name)

def get_embedding(sentence: str,):
  embeddings = model.encode(sentence)
  
  return embeddings

logger.debug(get_embedding(pages[0]['content']))

6582
'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
class_document =  {
  "class": "Livros",
  "description": "Dados do documento PDF",
  "moduleConfig": {
    "text2vec-transformers": {
      "poolingStrategy": "masked_mean",
      "vectorizeClassName": False
    }
  },
  "properties": [
    {
      "dataType": [
        "text"
      ],
      "description": "Conteúdo do documento PDF",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "content"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "source"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "page"
    }
  ],
  "vectorizer": "none"
}

client.schema.create_class(class_document)
# client.schema.property.create(class_name, {
#     'name': 'exemplar',
#     'dataType': ['text'],
# })

# client.schema.delete_class('Livros')
client.schema.get("Livros")

In [8]:
with client.batch as batch:
  batch.batch_size=100
  i = 0
    
  for page in pages:
    logger.info(f"importing question: {i+1}")
    i = i+1
    
    properties = {
            "content": page["content"],
            "page": str(page["page"]),
            "source": page["source"],
        }
    
    vector = get_embedding(page["content"])

    client.batch.add_data_object(properties, 'Livros', vector=vector)

2023-09-26 08:01:06,606 - loggingService - INFO - importing question: 1


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-09-26 08:01:07,053 - loggingService - INFO - importing question: 2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-09-26 08:01:07,440 - loggingService - INFO - importing question: 3


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-09-26 08:47:00,014 - loggingService - INFO - importing question: 6581


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-09-26 08:47:00,544 - loggingService - INFO - importing question: 6582


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def get_context(query: str, certainty= 0.8, limit = 4) -> str:
  """_summary_

  Args:
      query (str): _description_
      certainty (float, optional): _description_. Defaults to 0.8.
      limit (int, optional): _description_. Defaults to 4.
  """
  
  result = (client.query
  .get('Livros', ["content", "source", "page"])
  .with_additional(["certainty", "distance"]) # note that certainty is only supported if distance==cosine
  .with_near_vector({
    "vector": get_embedding(query),
    # "certainty": certainty
  })
  .with_limit(limit)
  .do()
  )
  
  print(result['data']['Get'][class_name])
  
  retorno = ''
  if len(result['data']['Get'][class_name]) == 0:
    return retorno
  
  retorno = result['data']['Get'][class_name][0]['content']
  
  for contexto in result['data']['Get'][class_name][1:]:
    retorno += f"\n{contexto['content']}"
  
  return retorno

logger.info(get_context('por que dom casmurro se chamava desse jeito', 0.2))
